In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
# from datasets import load_metric
# !pip install evaluate
# 
from sklearn.model_selection import train_test_split
import csv

import evaluate

In [2]:
label2id = {'negative':0,'positive':1}

train = pd.read_csv(r'./datasets/train.csv', encoding='utf_8_sig')
train = train.replace({"label": label2id})
train.fillna('', inplace=True)
test = pd.read_csv(r'./datasets/test.csv', encoding='utf_8_sig')
test = test.replace({"label": label2id})
val = pd.read_csv(r'./datasets/val.csv', encoding='utf_8_sig')
val = test.replace({"label": label2id})

test.fillna('', inplace=True)

In [3]:
train

,Unnamed: 0,label,message,diff
0,1385,0,Fix initialization of byte order in Creative V...,diff --git a/ChangeLog b/ChangeLog\nindex 11e7...
1,654,0,Fix stupid path error,diff --git a/captcha/simple-php-captcha.php b/...
2,1201,1,Fixed errors being dumped to the console when ...,diff --git a/examples/FilterShowcase/FilterSho...
3,2413,0,Update runtime files.,diff --git a/.github/CODEOWNERS b/.github/CODE...
4,4237,1,Add additional configuration options to the DI...,diff --git a/java/org/apache/catalina/authenti...
...,...,...,...,...
7073,5734,0,Fix getPicon,diff --git a/plugin/controllers/models/service...
7074,5191,0,HBASE-25361 [Flakey Tests] branch-2 TestMetaRe...,diff --git a/hbase-server/src/test/java/org/ap...
7075,5390,0,Update the meaning of low-hanging-fruit\n\nDur...,diff --git a/doc/source/contributor/how-can-i-...
7076,860,0,QPID-8368: [Broker-J] Make GrayLogger implemen...,diff --git a/broker-plugins/graylog-logging-lo...


In [4]:
len(train)

7078

In [5]:
train['label'].value_counts()

label
0    4478
1    2600
Name: count, dtype: int64

In [6]:
test['label'].value_counts()

label
0    926
1    591
Name: count, dtype: int64

In [7]:
from sklearn import  metrics

# def compute_metrics(y_pred, y_test):
#     # print(y_pred,y_test)
#     # classification_report = metrics.classification_report(y_test,y_pred)
#     accuracy_score = metrics.accuracy_score(y_test,y_pred)
#     recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
#     f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

#     # return {"classification_report": classification_report}
#     return {"accuracy": accuracy_score, "recall": recall_score, "f1":f1_score}

In [8]:
train

,Unnamed: 0,label,message,diff
0,1385,0,Fix initialization of byte order in Creative V...,diff --git a/ChangeLog b/ChangeLog\nindex 11e7...
1,654,0,Fix stupid path error,diff --git a/captcha/simple-php-captcha.php b/...
2,1201,1,Fixed errors being dumped to the console when ...,diff --git a/examples/FilterShowcase/FilterSho...
3,2413,0,Update runtime files.,diff --git a/.github/CODEOWNERS b/.github/CODE...
4,4237,1,Add additional configuration options to the DI...,diff --git a/java/org/apache/catalina/authenti...
...,...,...,...,...
7073,5734,0,Fix getPicon,diff --git a/plugin/controllers/models/service...
7074,5191,0,HBASE-25361 [Flakey Tests] branch-2 TestMetaRe...,diff --git a/hbase-server/src/test/java/org/ap...
7075,5390,0,Update the meaning of low-hanging-fruit\n\nDur...,diff --git a/doc/source/contributor/how-can-i-...
7076,860,0,QPID-8368: [Broker-J] Make GrayLogger implemen...,diff --git a/broker-plugins/graylog-logging-lo...


In [9]:
# id2label = {0: "Adaptive", 1: "POSITIVE"}
# label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "./models/bert-base-cased", num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained('./models/bert-base-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./models/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/root/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
training_args = TrainingArguments(
    output_dir="./my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

/root/miniconda3/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
encoded_train = tokenizer(train['message'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_test = tokenizer(test['message'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_val = tokenizer(val['message'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)

In [13]:
class CommitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [14]:
train_dataset = CommitDataset(encoded_train, list(train['label']))
test_dataset = CommitDataset(encoded_test, list(test['label']))
val_dataset = CommitDataset(encoded_val, list(test['label']))

In [15]:
train_dataset

In [16]:
def compute_metrics(eval_pred):
    accuracy = evaluate.load('./evaluate/metrics/accuracy')
    precision =  evaluate.load("./evaluate/metrics/precision")
    recall =  evaluate.load("./evaluate/metrics/recall")
    f1 =  evaluate.load("./evaluate/metrics/f1")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy.compute(predictions=predictions, references=labels)
    precision = precision.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall.compute(predictions=predictions, references=labels,average='weighted')
    f1 = f1.compute(predictions=predictions, references=labels,average='weighted')
    # accuracy_score = metrics.accuracy_score(labels,predictions)
    # precision_score = metrics.precision_score(labels,predictions, average='weighted')
    # recall_score = metrics.recall_score(labels,predictions,average='weighted')
    # f1_score = metrics.f1_score(labels,predictions,average='weighted')

    # return {"classification_report": classification_report}
    # return {"precision": precision_score,"recall": recall_score, "f1":f1_score, "accuracy": accuracy_score}
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [17]:
# !export https_proxy=http://turbo.gpushare.com:30000 http_proxy=http://turbo.gpushare.com:30000
# !export https_proxy=http://turbo2.gpushare.com:30000 http_proxy=http://turbo2.gpushare.com:30000

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    # model_init= model_init,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)
trainer.train()
# best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, compute_objective=my_compute_objective, n_trials=10)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/tmp/ipykernel_1143/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.352852,{'precision': 0.8523298172683502},{'recall': 0.8529993408042189},{'f1': 0.8524939601912157},{'accuracy': 0.8529993408042189}


/tmp/ipykernel_1143/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Trainer is attempting to log a value of "{'precision': 0.8523298172683502}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.8529993408042189}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.8524939601912157}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is a

TrainOutput(global_step=222, training_loss=0.46053431055567284, metrics={'train_runtime': 185.1973, 'train_samples_per_second': 38.219, 'train_steps_per_second': 1.199, 'total_flos': 1862300049838080.0, 'train_loss': 0.46053431055567284, 'epoch': 1.0})

In [19]:
# best_run.hyperparameters

In [20]:
# trainer.apply_hyperparameters(best_run.hyperparameters, final_model=True)
# trainer.train()

In [21]:
fewshot_metrics = trainer.evaluate(eval_dataset=test_dataset)
fewshot_metrics

/tmp/ipykernel_1143/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Trainer is attempting to log a value of "{'precision': 0.8523298172683502}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.8529993408042189}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.8524939601912157}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.8529993408042189}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.35106971859931946,
 'eval_precision': {'precision': 0.8523298172683502},
 'eval_recall': {'recall': 0.8529993408042189},
 'eval_f1': {'f1': 0.8524939601912157},
 'eval_accuracy': {'accuracy': 0.8529993408042189},
 'eval_runtime': 12.6527,
 'eval_samples_per_second': 119.895,
 'eval_steps_per_second': 3.794,
 'epoch': 1.0}